In [1]:
import os
from qdrant_client import QdrantClient
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http.models import Distance, VectorParams
from langchain_cohere import CohereEmbeddings
from langchain.schema import Document

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [84]:
# # read the document
# def read_doc(directory):
#     file_loader=PyPDFDirectoryLoader(directory)
#     documents=file_loader.load()
#     return documents

In [ ]:
# doc=read_doc('documents/')
# len(doc)

In [86]:
# ## Divide the docs into chunks
# def chunk_data(docs,chunk_size=800,chunk_overlap=50):
#     text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
#     doc=text_splitter.split_documents(docs)
#     return doc

In [3]:
# documents=chunk_data(docs=doc)
# documents
user_profiles = [
    {
      "Name": "Emily Davis",
      "Age": 29,
      "Location": "New York, USA",
      "Gender": "Female",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Photography", "Traveling", "Yoga"],
      "Likes": ["Italian food", "Beaches", "Jazz music"],
      "Dislikes": ["Crowded places", "Fast food"],
      "Occupation": "Marketing Manager",
      "Hobbies": ["Painting", "Reading fiction"],
      "Looking For": "Serious relationship"
    },
    {
      "Name": "James Miller",
      "Age": 34,
      "Location": "Los Angeles, USA",
      "Gender": "Male",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Hiking", "Sports", "Technology"],
      "Likes": ["Basketball", "Sci-fi movies", "Mexican food"],
      "Dislikes": ["Loud music", "Spicy food"],
      "Occupation": "Software Engineer",
      "Hobbies": ["Gaming", "Coding"],
      "Looking For": "Casual dating"
    },
    {
      "Name": "Sophia Johnson",
      "Age": 26,
      "Location": "London, UK",
      "Gender": "Female",
      "Sexual Orientation": "Bisexual",
      "Interests": ["Fashion", "Music", "Social media"],
      "Likes": ["Shopping", "Pop music", "Coffee"],
      "Dislikes": ["Waking up early", "Rainy weather"],
      "Occupation": "Fashion Blogger",
      "Hobbies": ["Singing", "Dancing"],
      "Looking For": "Serious relationship"
    },
    {
      "Name": "Michael Anderson",
      "Age": 31,
      "Location": "Toronto, Canada",
      "Gender": "Male",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Traveling", "Food", "Cars"],
      "Likes": ["Road trips", "Burgers", "Football"],
      "Dislikes": ["Cold weather", "Long meetings"],
      "Occupation": "Sales Executive",
      "Hobbies": ["Photography", "Cycling"],
      "Looking For": "Long-term relationship"
    },
    {
      "Name": "Ava Martinez",
      "Age": 25,
      "Location": "Barcelona, Spain",
      "Gender": "Female",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Art", "History", "Travel"],
      "Likes": ["Museums", "Classical music", "Tapas"],
      "Dislikes": ["Fast fashion", "Crowded places"],
      "Occupation": "Museum Curator",
      "Hobbies": ["Sketching", "Playing piano"],
      "Looking For": "Serious relationship"
    },
    {
      "Name": "Daniel Kim",
      "Age": 28,
      "Location": "Seoul, South Korea",
      "Gender": "Male",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Technology", "Gaming", "Cooking"],
      "Likes": ["K-pop", "Street food", "Anime"],
      "Dislikes": ["Hot weather", "Public speaking"],
      "Occupation": "App Developer",
      "Hobbies": ["Coding", "Martial arts"],
      "Looking For": "Friendship"
    },
    {
      "Name": "Olivia Brown",
      "Age": 27,
      "Location": "Sydney, Australia",
      "Gender": "Female",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Fitness", "Beach", "Movies"],
      "Likes": ["Surfing", "Coffee", "Action films"],
      "Dislikes": ["Spiders", "Crowds"],
      "Occupation": "Personal Trainer",
      "Hobbies": ["Yoga", "Running"],
      "Looking For": "Casual dating"
    },
    {
      "Name": "Liam Wilson",
      "Age": 30,
      "Location": "Dublin, Ireland",
      "Gender": "Male",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Sports", "Reading", "History"],
      "Likes": ["Soccer", "Irish music", "Craft beer"],
      "Dislikes": ["Laziness", "Pessimism"],
      "Occupation": "History Teacher",
      "Hobbies": ["Playing guitar", "Fishing"],
      "Looking For": "Long-term relationship"
    },
    {
      "Name": "Emma White",
      "Age": 24,
      "Location": "Vancouver, Canada",
      "Gender": "Female",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Nature", "Hiking", "Animals"],
      "Likes": ["Vegan food", "Sunsets", "Documentaries"],
      "Dislikes": ["Pollution", "Loud noises"],
      "Occupation": "Veterinarian",
      "Hobbies": ["Photography", "Volunteering"],
      "Looking For": "Serious relationship"
    },
    {
      "Name": "Noah Thompson",
      "Age": 35,
      "Location": "Berlin, Germany",
      "Gender": "Male",
      "Sexual Orientation": "Homosexual",
      "Interests": ["Tech", "Art", "Music"],
      "Likes": ["Electronic music", "Coffee", "Biking"],
      "Dislikes": ["Smoking", "Heavy metal music"],
      "Occupation": "Graphic Designer",
      "Hobbies": ["Painting", "DJing"],
      "Looking For": "Serious relationship"
    },
    {
      "Name": "Isabella Garcia",
      "Age": 29,
      "Location": "Buenos Aires, Argentina",
      "Gender": "Female",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Cooking", "Travel", "Languages"],
      "Likes": ["Tango", "Wine", "Beach"],
      "Dislikes": ["Dishonesty", "Long flights"],
      "Occupation": "Chef",
      "Hobbies": ["Baking", "Learning new languages"],
      "Looking For": "Friendship"
    },
    {
      "Name": "Alexander Lee",
      "Age": 32,
      "Location": "Hong Kong",
      "Gender": "Male",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Finance", "Cars", "Technology"],
      "Likes": ["Luxury cars", "Sushi", "Movies"],
      "Dislikes": ["Arrogance", "Waiting"],
      "Occupation": "Investment Banker",
      "Hobbies": ["Golf", "Traveling"],
      "Looking For": "Long-term relationship"
    },
    {
      "Name": "Mia Patel",
      "Age": 24,
      "Location": "Mumbai, India",
      "Gender": "Female",
      "Sexual Orientation": "Bisexual",
      "Interests": ["Yoga", "Meditation", "Spirituality"],
      "Likes": ["Indian cuisine", "Folk music", "Temples"],
      "Dislikes": ["Pollution", "Overcrowding"],
      "Occupation": "Yoga Instructor",
      "Hobbies": ["Dancing", "Traveling"],
      "Looking For": "Serious relationship"
    },
    {
      "Name": "Benjamin Carter",
      "Age": 36,
      "Location": "Auckland, New Zealand",
      "Gender": "Male",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Fishing", "Adventure sports", "Nature"],
      "Likes": ["Mountain biking", "Seafood", "Camping"],
      "Dislikes": ["Traffic", "Long meetings"],
      "Occupation": "Entrepreneur",
      "Hobbies": ["Rock climbing", "Kayaking"],
      "Looking For": "Friendship"
    },
    {
      "Name": "Chloe Schmidt",
      "Age": 27,
      "Location": "Zurich, Switzerland",
      "Gender": "Female",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Skiing", "Photography", "Food"],
      "Likes": ["Swiss chocolate", "Mountains", "Wine"],
      "Dislikes": ["Early mornings", "Cold food"],
      "Occupation": "Architect",
      "Hobbies": ["Drawing", "Hiking"],
      "Looking For": "Serious relationship"
    },
    {
      "Name": "Jack Adams",
      "Age": 31,
      "Location": "Chicago, USA",
      "Gender": "Male",
      "Sexual Orientation": "Heterosexual",
      "Interests": ["Finance", "Movies", "Fitness"],
      "Likes": ["Gym", "Action movies", "Pizza"],
      "Dislikes": ["Messy places", "Procrastination"],
      "Occupation": "Financial Analyst",
      "Hobbies": ["Running", "Basketball"],
      "Looking For": "Casual dating"
    }
]

In [4]:
# Create function to convert profile to text
def profile_to_text(profile):
    interests = ", ".join(profile.get("Interests", []))
    likes = ", ".join(profile.get("Likes", []))
    dislikes = ", ".join(profile.get("Dislikes", []))
    hobbies = ", ".join(profile.get("Hobbies", []))
    
    return (
        f"Name: {profile['Name']}, Age: {profile['Age']}, "
        f"Location: {profile['Location']}, Gender: {profile['Gender']}, "
        f"Sexual Orientation: {profile['Sexual Orientation']}, "
        f"Occupation: {profile['Occupation']}, "
        f"Interests: {interests}, Likes: {likes}, Dislikes: {dislikes}, "
        f"Hobbies: {hobbies}, Looking For: {profile['Looking For']}"
    )

In [5]:
documents = [Document(page_content=profile_to_text(profile)) for profile in user_profiles]
documents

[Document(page_content='Name: Emily Davis, Age: 29, Location: New York, USA, Gender: Female, Sexual Orientation: Heterosexual, Occupation: Marketing Manager, Interests: Photography, Traveling, Yoga, Likes: Italian food, Beaches, Jazz music, Dislikes: Crowded places, Fast food, Hobbies: Painting, Reading fiction, Looking For: Serious relationship'),
 Document(page_content='Name: James Miller, Age: 34, Location: Los Angeles, USA, Gender: Male, Sexual Orientation: Heterosexual, Occupation: Software Engineer, Interests: Hiking, Sports, Technology, Likes: Basketball, Sci-fi movies, Mexican food, Dislikes: Loud music, Spicy food, Hobbies: Gaming, Coding, Looking For: Casual dating'),
 Document(page_content='Name: Sophia Johnson, Age: 26, Location: London, UK, Gender: Female, Sexual Orientation: Bisexual, Occupation: Fashion Blogger, Interests: Fashion, Music, Social media, Likes: Shopping, Pop music, Coffee, Dislikes: Waking up early, Rainy weather, Hobbies: Singing, Dancing, Looking For: Se

In [6]:
embeddings_model = CohereEmbeddings(cohere_api_key=os.environ['COHERE_API_KEY'], model='embed-english-v3.0')
# embeddings_model

In [9]:
# embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
# len(embedded_query)
# Generate embeddings for each user profile
embeddings = embeddings_model.embed_documents([doc.page_content for doc in documents])

In [7]:
qdrant_client = QdrantClient(
    os.environ['QDRANT_HOST'],
    api_key=os.environ['QDRANTDB_API_KEY'],
)

print(qdrant_client.get_collections())

collections=[]


In [8]:
vector_config = VectorParams(
    size=1024,
    distance=Distance.COSINE
)

qdrant_client.create_collection(
    collection_name=os.environ['COLLECTION_NAME'],
    vectors_config=vector_config
)

True

In [9]:
from uuid import uuid4


vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=os.environ['COLLECTION_NAME'],
    embedding=embeddings_model,
)

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(
    documents=documents, 
    ids=uuids
)

['da01ed50-2714-4884-9025-dfde9ff4a210',
 '372c1f0f-e50a-4b7a-9443-485f6c8b5934',
 'f2e3d9d9-510a-4b48-a6a3-4df4270f2c15',
 '58c683cf-b253-4323-9c62-694eaf426f9d',
 '0e1a30fd-1a59-486e-9660-796d8db4c2e1',
 '976e9c33-0a1d-46c6-a44b-95f00b245137',
 '03070b1d-ff94-4cf2-a6f4-1f5dc96a688f',
 '639ee640-ed43-4434-a93d-0d76ae9cf80d',
 '40ede169-51b6-4597-a730-12a6eb484f36',
 '47bc8c58-f305-4f75-812f-6290d0e0ff23',
 'a7dc6112-52f4-4a5c-8aef-5c1c99daef2e',
 'b2b0ce49-644b-4327-80e1-ad62bef05e15',
 '1a8e6117-b1de-4e86-b573-1710ed413c41',
 'c8d38ced-5f26-41c4-b64b-04342055bb09',
 '2d90d755-75ce-4b65-8a46-2fdd1af5c808',
 '8f24bdbb-5009-4b64-a780-5b91fed1491c']

In [ ]:
query = 'Name: James Miller, Age: 34, Location: Los Angeles, USA, Gender: Male, Sexual Orientation: Heterosexual, Occupation: Software Engineer, Interests: Hiking, Sports, Technology, Likes: Basketball, Sci-fi movies, Mexican food, Dislikes: Loud music, Spicy food, Hobbies: Gaming, Coding, Looking For: Casual dating'
results = vector_store.similarity_search_with_score(query, k=5)
results